필요한 기능들 import

In [40]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2
import PIL
import gc
import psutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
from math import ceil
import math
import sys
import gc

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import applications
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping

from keras.activations import softmax
from keras.activations import elu
from keras.activations import relu
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization #정규화
# from tensorflow.keras.layers.GroupNormalization import BatchNormalization
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
#from keras.layers.normalization import BatchNormalization

# 이미지 load하고 픽셀로 바꿈

In [41]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

data_dir = "./data/train"
categories = ["melanoma", "nevus", "seborrheic_keratosis"]
n_classes = len(categories)

#이미지크기
image_w = 224 
image_h = 224

#pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(n_classes)]
    label[idx] = 1
    image_dir = data_dir + "/" + cat
    
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0: #중간확인출력
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save('./numpy_data/multi_data.npy', xy)

print("ok", len(y))

melanoma  파일 길이 :  374
melanoma  :  ./data/train/melanoma\ISIC_0000002.jpg
nevus  파일 길이 :  1372
nevus  :  ./data/train/nevus\ISIC_0000000.jpg
nevus  :  ./data/train/nevus\ISIC_0010558.jpg
seborrheic_keratosis  파일 길이 :  254
seborrheic_keratosis  :  ./data/train/seborrheic_keratosis\ISIC_0012090.jpg
ok 2000


In [42]:
X_train, X_test, Y_train, Y_test = np.load('./numpy_data/multi_data.npy', allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

(1500, 224, 224, 3)
1500


**----------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [6]:
# df = pd.read_pickle('/content/drive/My Drive/CS 412 PROJECT/data/data224x224.csv')
# #df['image'] = df['image'].map(lambda img: img/255)
# df.head()

In [7]:
#print(df['image'][0].shape)

In [8]:
# #features and labels are splitted
# X = df['image']
# y = df['Category']

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
#train-validation split
from keras.utils import np_utils
trainx, valx, trainy, valy = train_test_split(X, y, test_size=0.03,random_state=41)
testx = valx
testy = valy

In [11]:
# #turning image vectors into arrays
# trainx = np.asarray(trainx.tolist())
# testx = np.asarray(testx.tolist())
# valx = np.asarray(valx.tolist())

In [12]:
#encoding the categories using one hot encoding
# print("Shape before one-hot encoding: ", trainy.shape)
# trainy = np_utils.to_categorical(trainy, 3)
# testy = np_utils.to_categorical(testy, 3)
# valy = np_utils.to_categorical(valy, 3)
# print("Shape after one-hot encoding: ", trainy.shape)

Shape before one-hot encoding:  (113, 3)
Shape after one-hot encoding:  (113, 3, 3)


In [45]:
print("X_train shape", trainx.shape)
print("y_train shape", trainy.shape)
print("X_val shape", valx.shape)
print("y_val shape", valy.shape)
print("X_test shape", testx.shape)
print("y_test shape", testy.shape)

X_train shape (1940, 224, 224, 3)
y_train shape (1940, 3)
X_val shape (60, 224, 224, 3)
y_val shape (60, 3)
X_test shape (60, 224, 224, 3)
y_test shape (60, 3)


In [46]:
#reshaping the arrays to fit the models
trainx = trainx.reshape(trainx.shape[0], *(224, 224, 3))
testx = testx.reshape(testx.shape[0], *(224, 224, 3))
valx = valx.reshape(valx.shape[0], *(224, 224, 3))

In [47]:
from keras.applications.resnet import preprocess_input

#choosing the generator and the differences that would happen in the generated images
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=1./255,
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.5,1.5],
    zoom_range=0.6,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

#choosing the generator and the differences that would happen in the generated images
test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input,
    brightness_range=[0.5,1.5])


#generating additional data
train_generator = train_datagen.flow(
    trainx,
    y=trainy,
    batch_size=32,
    shuffle=True,
    seed=42
    )

#generating additional data
valid_generator=test_datagen.flow(
    valx,
    y=valy,
    batch_size=32,
    shuffle=True,
    seed=42
    )

In [49]:
from tensorflow.keras.applications.resnet50 import ResNet50

input_shape = (224,224,3)
lr = 1e-5
epochs = 10
batch_size = 16

base_model = ResNet50(include_top=False, input_shape= (224,224,3), weights = 'imagenet') #base model

x = base_model.output

x = GlobalAveragePooling2D()(x) #extra layers
x = Dropout(0.35)(x) #to avoid overfitting
predictions = Dense(3, activation= 'softmax')(x) #output dense layer, we used softmax because it is multiclass
model = Model(inputs = base_model.input, outputs = predictions)


model.compile(optimizer = Adam(lr) ,
              loss = "categorical_crossentropy", 
              metrics=["accuracy"]) #this is the comiler we used Adam with a given learning rate

model_path = './model' + '/multi_img_classification.model'
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, mode='max') #this is the function that saves the best epoch

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, 
                                   verbose=1, mode='max', min_lr=0.00001) #another overfitting measure
                              
                              
callbacks_list = [checkpoint, reduce_lr]

print("X_train shape", X_train.shape)
print("y_train shape", Y_train.shape)
print("X_train shape", trainx.shape)
print("y_train shape", trainy.shape)

history = model.fit(x=X_train, y=Y_train,
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(valx,valy), 
                    callbacks=[checkpoint])

# history = model.fit_generator(train_generator,
#                               steps_per_epoch = 600,
#                               epochs = epochs,
#                               validation_data = valid_generator,
#                               callbacks= callbacks_list) #training of the model

X_train shape (1500, 224, 224, 3)
y_train shape (1500, 3)
X_train shape (1940, 224, 224, 3)
y_train shape (1940, 3)
Epoch 1/10
94/94 [==============================] - ETA: 0s - loss: 0.9123 - accuracy: 0.6093
Epoch 00001: val_accuracy improved from -inf to 0.78333, saving model to ./model\multi_img_classification.model
INFO:tensorflow:Assets written to: ./model\multi_img_classification.model\assets


C:\Users\User\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\User\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


94/94 [==============================] - 443s 5s/step - loss: 0.9123 - accuracy: 0.6093 - val_loss: 0.7776 - val_accuracy: 0.7833
Epoch 2/10
94/94 [==============================] - ETA: 0s - loss: 0.6457 - accuracy: 0.7433
Epoch 00002: val_accuracy did not improve from 0.78333
94/94 [==============================] - 441s 5s/step - loss: 0.6457 - accuracy: 0.7433 - val_loss: 0.5216 - val_accuracy: 0.7833
Epoch 3/10
94/94 [==============================] - ETA: 0s - loss: 0.5201 - accuracy: 0.7880
Epoch 00003: val_accuracy improved from 0.78333 to 0.88333, saving model to ./model\multi_img_classification.model
INFO:tensorflow:Assets written to: ./model\multi_img_classification.model\assets


C:\Users\User\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\User\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


94/94 [==============================] - 476s 5s/step - loss: 0.5201 - accuracy: 0.7880 - val_loss: 0.3585 - val_accuracy: 0.8833
Epoch 4/10
94/94 [==============================] - ETA: 0s - loss: 0.4076 - accuracy: 0.8493
Epoch 00004: val_accuracy improved from 0.88333 to 0.90000, saving model to ./model\multi_img_classification.model
INFO:tensorflow:Assets written to: ./model\multi_img_classification.model\assets


...

94/94 [==============================] - 462s 5s/step - loss: 0.2550 - accuracy: 0.9133 - val_loss: 0.2058 - val_accuracy: 0.9500
Epoch 7/10
94/94 [==============================] - ETA: 0s - loss: 0.1906 - accuracy: 0.9420 
Epoch 00007: val_accuracy did not improve from 0.95000
94/94 [==============================] - 2103s 23s/step - loss: 0.1906 - accuracy: 0.9420 - val_loss: 0.1891 - val_accuracy: 0.9500
Epoch 8/10
94/94 [==============================] - ETA: 0s - loss: 0.1494 - accuracy: 0.9613
Epoch 00008: val_accuracy did not improve from 0.95000
94/94 [==============================] - 454s 5s/step - loss: 0.1494 - accuracy: 0.9613 - val_loss: 0.1911 - val_accuracy: 0.9500
Epoch 9/10
94/94 [==============================] - ETA: 0s - loss: 0.1154 - accuracy: 0.9760
Epoch 00009: val_accuracy improved from 0.95000 to 0.96667, saving model to ./model\multi_img_classification.model
INFO:tensorflow:Assets written to: ./model\multi_img_classification.model\assets


C:\Users\User\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


94/94 [==============================] - 491s 5s/step - loss: 0.1154 - accuracy: 0.9760 - val_loss: 0.1854 - val_accuracy: 0.9667


C:\Users\User\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 10/10
94/94 [==============================] - ETA: 0s - loss: 0.0977 - accuracy: 0.9833
Epoch 00010: val_accuracy did not improve from 0.96667
94/94 [==============================] - 478s 5s/step - loss: 0.0977 - accuracy: 0.9833 - val_loss: 0.1769 - val_accuracy: 0.9500
